In [3]:
# Imports

from random import shuffle
import csv
import re

In [4]:
# Make input lists

startOrder = ['B', 'B', 'B', 'G', 'G', 'G', 'Y', 'Y', 'Y', 'R', 'R', 'R', 'D']

needOrder = startOrder
while (needOrder[0] == 'D' or needOrder[-1] == 'D'):
    shuffle(needOrder)

loveOrder = startOrder
while (loveOrder[0] == 'D' or loveOrder[-1] == 'D'):
    shuffle(loveOrder)
    
needOrder

['B', 'R', 'G', 'B', 'G', 'Y', 'D', 'Y', 'G', 'R', 'Y', 'B', 'R']

In [5]:
# Read a CSV to a list

def readCSV(filename):
    with open(filename, 'rb') as f:
        reader = csv.reader(f)
        your_list = list(reader)
    return your_list

In [6]:
game = readCSV("game.csv")

dictCards = {}
for turn in game:
    for color in turn:
        #regexp = re.compile(r'\*[BGYRD]\*')
        regexp = re.compile(r'[BGYRD]\*\*')
        if regexp.search(color):
            if color in dictCards:
                dictCards[color] += 1
            else:
                dictCards[color] = 0

for turn in game:
    for color in turn:
        #regexp = re.compile(r'\*[BGYRD]\*')
        regexp = re.compile(r'\*\*[BGYRD]')
        if regexp.search(color):
            if color in dictCards:
                dictCards[color] += 1
            else:
                dictCards[color] = 0

print dictCards
print dictCards['**D'] + dictCards['D**']
print dictCards['**R'] + dictCards['R**']
print dictCards['**Y'] + dictCards['Y**']
print dictCards['**B'] + dictCards['B**']
print dictCards['**G'] + dictCards['G**']

{'D**': 1, '**Y': 3, 'Y**': 1, 'B**': 2, '**R': 2, 'G**': 1, '**G': 2, '**D': 1, 'R**': 1, '**B': 0}
2
3
4
2
3


In [7]:
# Play a card onto the table
# in - array of cards on the table
# in - card just played
# in - position that the card was played at
# out - updated cards on table

def playCard(playedCards, currCard, position):
    
    cardColor = currCard[1]
    cardLeft = currCard[2]
    cardMiddle = currCard[3]
    cardRight = currCard[4]
    playedCards.insert(position, currCard)
    
    oldPoints = playedCards
    
    # Add points to the played card depending on the cards beside it
    if (len(playedCards) > 1):
        if (position > 0):
            if (playedCards[position - 1][1] == cardLeft):
                if (cardLeft == 'D'):
                    playedCards[position - 1][0] += 1
                else:
                    playedCards[position][0] += oldPoints[position - 1][0]
        if (position < len(playedCards) - 1):
            if (playedCards[position + 1][1] == cardRight):
                if (cardRight == 'D'):
                    playedCards[position + 1][0] += 1
                else:
                    playedCards[position][0] += oldPoints[position + 1][0]
    
    # Go through each card left to right
    for i, card in enumerate(playedCards):
        
        # See if the middle of the card is the card just added
        if (card[1] == cardMiddle):
            card[0] += 1
            
        # If we are looking at the card to the left of the
        # card just played
        if (i == position - 1):
            if (playedCards[i][4] == cardColor):
                if (cardColor == 'D'):
                    playedCards[position] += 1
                else:
                    card[0] += oldPoints[i][0]
        
        # If we are looking at the card to the right of the
        # card just played
        if (i == position + 1):
            if (playedCards[i][2] == cardColor):
                if (cardColor == 'D'):
                    playedCards[position] += 1
                else:
                    card[0] += oldPoints[i][0]
                
    return playedCards

In [8]:
# Verify test

def unitTest(result, verify):
    if (not result == verify):
        raise ValueError('Test Failed.\n' + str(result) + '\n!=\n' + str(verify))

In [16]:
# Unit tests

test1 = playCard([], [1, 'R', 'N', 'N', 'G'], 0)
test1 = playCard(test1, [1, 'B', 'R', 'N', 'N'], 1)
test1 = playCard(test1, [1, 'G', 'B', 'N', 'R'], 0)
test1 = playCard(test1, [1, 'Y', 'N', 'N', 'B'], 2)

test1Solution = [[2, 'G', 'B', 'N', 'R'], [1, 'R', 'N', 'N', 'G'], [3, 'Y', 'N', 'N', 'B'], [2, 'B', 'R', 'N', 'N']]
unitTest(test1, test1Solution)

test2 = playCard([], [-3, 'D', 'N', 'N', 'N'], 0)
test2 = playCard(test2, [1, 'R', 'D', 'N', 'N'], 1)
test2 = playCard(test2, [1, 'Y', 'R', 'N', 'N'], 2)
test2 = playCard(test2, [1, 'G', 'N', 'B', 'N'], 1)

test2Solution = [[-2, 'D', 'N', 'N', 'N'], [1, 'G', 'N', 'B', 'N'], [1, 'R', 'D', 'N', 'N'], [2, 'Y', 'R', 'N', 'N']]
unitTest(test2, test2Solution)

test3 = playCard([], [1, 'Y', 'Y', 'N', 'B'], 0)
test3 = playCard(test3, [1, 'B', 'N', 'N', 'Y'], 0)
test3 = playCard(test3, [1, 'G', 'Y', 'N', 'N'], 2)
test3 = playCard(test3, [-3, 'D', 'N', 'N', 'N'], 3)
print test3

test3Solution = [[-2, 'D', 'N', 'N', 'N'], [1, 'G', 'N', 'B', 'N'], [1, 'R', 'D', 'N', 'N'], [2, 'Y', 'R', 'N', 'N']]
unitTest(test3, test2Solution)

[[2, 'B', 'N', 'N', 'Y'], [1, 'Y', 'Y', 'N', 'B'], [2, 'G', 'Y', 'N', 'N'], [-3, 'D', 'N', 'N', 'N']]


ValueError: Test Failed.
[[2, 'B', 'N', 'N', 'Y'], [1, 'Y', 'Y', 'N', 'B'], [2, 'G', 'Y', 'N', 'N'], [-3, 'D', 'N', 'N', 'N']]
!=
[[-2, 'D', 'N', 'N', 'N'], [1, 'G', 'N', 'B', 'N'], [1, 'R', 'D', 'N', 'N'], [2, 'Y', 'R', 'N', 'N']]